# Script to update inp files for new simulation
This script is a basic version of da_inp.m, it does not contain all options, e.g. trees etc. are missing. See da_inp.m for information on how to implement it.

# Contents
[Setup](#Setup)

[Update parameters in namoptions](#Update-parameters-in-namoptions)

[Update grids](#Update-grids)

[Update forcing](#Update-forcing)

[Update blocks](#Update-blocks)

# Setup

##### Set up libaries, paths and data

In [1]:
%matplotlib inline
import sys, os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pprint
sys.path.append('predales')
import predales as da

##### Set the paths

In [ ]:
# set paths
projectpath, exppath, workpath, ephemeral = da.tools.environment()  # needs to be adapted if other people use it
# set preprocessing directory
predir = projectpath + '/pre-post/pre/'
# define standard colors
light, dark, emphcolors = da.tools.colorschemes()
# define additional plot styles
linestyles = da.tools.plotstyles()
# define the plot style
# Can be configured in os.path.join(mpl._get_configdir(), 'stylelib').
# Default styles are in os.path.join(mpl.get_data_path(), 'stylelib').
mpl.style.use(dark)
# mpl.style.use(light)

---
## Load base configuration

#### _Set the experiment number:_

In [ ]:
expnr = '593'

##### Get current numoptions

In [ ]:
# get namoptions file
inpspath = os.path.join(exppath, expnr, '')
namoptions = inpspath + 'namoptions.' + expnr
# get namoptions data
parameters = da.tools.getparameters(namoptions)
# extract to searchable parameter dictionary
paralist = da.tools.extractparameters(parameters)
# print current parameters
pprint.pprint(parameters)

[Back to Contents](#Contents)

---
# Update parameters in namoptions

In [ ]:
changes = {'iexpnr' : int(expnr), 'runtime': 100004, 
           'tfielddump' : 50000,  'tstatsdump' : 10000, 'trestart' : 10000,
           'lydump' : False, 'lytdump' : False}
parameters = da.write.namoptions(parameters, changes, namoptions)
paralist = da.tools.extractparameters(parameters)  # update paralist
# pprint.pprint(parameters)

[Back to Contents](#Contents)

---
# Update grids

## Domain size values

In [ ]:
# ## set default values
# xsize = 480
# ysize = 240
# zsize = 180
# imax = 240
# jtot = 120
# kmax = 180

## get domain parameters
paralist = da.tools.extractparameters(parameters)
xsize = paralist['xsize']
ysize = paralist['ysize']
# zsize = paralist['zsize']
zsize = 180
imax = paralist['imax']
jtot = paralist['jtot']
kmax = paralist['kmax']
print("xsize: ", xsize, " imax: ", imax)
print("ysize: ", ysize, " jtot: ", jtot)
print("zsize: ", zsize, " kmax: ", kmax)

In [ ]:
# new grid for Dales code with surface street
gridstretching = True

# Processing x-grid
dx = xsize/imax
xedge = np.arange(0, xsize, dx)
xcentre = xedge + dx*0.5
xgrid = xcentre

# Processing y-grid (for plotting only)
dy = ysize/jtot
yedge = np.arange(0, ysize, dy)
ycentre = yedge + dy*0.5
ygrid = ycentre

# Processing z-grid    
dz = zsize/kmax

if gridstretching is True:
    # Stretch z-grid from n onwards with expansion ratio r
    n = round(kmax/2)  # stretch from this index on
    r = 1.01  # expansion ratio between stretched cells
    
    stretch = np.concatenate((np.zeros(n),np.arange(kmax - n)))
    expand = [r**i for i in stretch]
    dzs = [dz*si for si in expand]
    
    zedge = np.zeros(kmax)
    for i, dzi in enumerate(dzs[:-1]):
        zedge[i + 1] = zedge[i] + dzi
    zcentre = [ze + dzi*0.5 for ze, dzi in zip(zedge, dzs)]
    
    newzsize = round((zedge[-1] + dzs[-1]) - zedge[0], 4)

else:
    zedge = np.arange(0, zsize, dz)
    # zedge = np.arange(-dz, zsize-dz, dz)
    zcentre = zedge + dz*0.5
    # zsize -= dz
    dzs = [dz] * kmax
    
    newzsize = zsize

zgrid = zcentre

# new z grid
print("zsize:\n", newzsize)
print("zm:\n", zedge)
print("zt:\n", zcentre)
# plot the grid, every 10th element
for z in zgrid[::10]:
    plt.axhline(y=z)
plt.show()

#### Write new grids to inp file

In [ ]:
# write grid files
da.write.gridinp(xgrid, inpspath + 'xgrid.inp.' + expnr, gridname='x-grid')
da.write.gridinp(zgrid, inpspath + 'zgrid.inp.' + expnr, gridname='z-grid')
# update parameters in namoptions
gridchanges = {'xsize': xsize, 'ysize': ysize, 'zsize' : newzsize,
               'imax': imax, 'jtot': jtot, 'kmax': kmax}
parameters = da.write.namoptions(parameters, gridchanges, namoptions)


[Back to Contents](#Contents)

---
# Update forcing

## Determine forcing type

In [ ]:
check = 0
# # set default values
u0 = 2  # inital velocities
v0 = 0  
# dpdx = 0.0005  # pressure gradients
# dpdy = 0
# get initial forcing parameters
# u0 = parameters['inps']['u0']  # inital velocities
# v0 = parameters['inps']['v0']
# dpdx = parameters['inps']['dpdx']  # pressure gradients
# dpdy = parameters['inps']['dpdy']

if (parameters['run']['luflowr'] is True) or (parameters['run']['lvflowr'] is True):
    check += 1
    print("Forcing is constant volume flow rate.")
    # pressure difference
    pqx = 0  
    pqy = 0
    # geostrophic wind
    ug = 0  
    vg = 0
    # initial velocity
    if (parameters['run']['luflowr'] is True) and (parameters['run']['lvflowr'] is True):
        u = u0  
        v = v0
    elif parameters['run']['luflowr'] is True:
        u = u0
        v = 0
    elif parameters['run']['lvflowr'] is True:
        u = 0
        v = v0

if parameters['physics']['lprofforc'] is True:
    check += 1
    print("Forcing is 1D geostrophic forcing.")
    # pressure difference
    pqx = 0 
    pqy = 0
    # geostrophic wind
    ug = u0 
    vg = v0
    # initial velocity
    u = u0
    v = v0

if parameters['physics']['lcoriol'] is True:
    check += 1
    print("Forcing is coriolis forcing.")
    # pressure difference
    pqx = 0 
    pqy = 0
    # geostrophic wind
    ug = v0  # really??
    vg = u0  # really??
    # initial velocity
    u = u0
    v = v0

if check == 0:
    check += 1
    print("Forcing is pressure gradient forcing.")
    # pressure difference
    pqx = dpdx
    pqy = dpdy
    # geostrophic wind
    ug = 0
    vg = 0
    # initial velocity
    u = u0
    v = v0
    
if check > 1:
    raise ValueError("More than one forcing specified.")

print("Pressure gradients are:")
print("dp/dx:", pqx)
print("dp/dy:", pqy)
print("Geostrophic wind velocities are:")
print("ug:", ug)
print("vg:", vg)
print("Initial wind velocities are:")
print("u0:", u)
print("v0:", v)

#### Update parameters in namoptions file

In [ ]:
# update parameters in namoptions
forcechanges = {'u0': u, 'v0': v, 'dpdx': pqx, 'dpdy': pqy}
parameters = da.write.namoptions(parameters, forcechanges, namoptions)

## Lscale

In [ ]:
# set default values
ws = 0  # subsidence
dthlrad = 0  # radiative forcing
dqtdxls = 0
dqtdyls = 0
dqtdtls = 0
# ws = parameters['inps']['w_s']  # subsidence
# dthlrad = parameters['inps']['R']  # radiative forcing

lscales = [[z, ug, vg, pqx, pqy, ws, dqtdxls, dqtdyls, dqtdtls, dthlrad] for z in zgrid]

#### Write new lscale inp file

In [ ]:
da.write.lscaleinp(lscales, inpspath + 'lscale.inp.' + expnr)

#### Update parameters in namoptions file

In [ ]:
lscalechanges = {'w_s': ws, 'R': dthlrad}
parameters = da.write.namoptions(parameters, lscalechanges, namoptions)

## Prof

In [ ]:
# set default values
thl0 = 288  # temperature at ground level
lapse = 0  # lapse rate Ks-1
qt0 = 0
tke = 0
# thl0 = parameters['inps']['thl0']  # temperature at ground level
# lapse = parameters['inps']['lapse']  # lapse rate Ks-1

if abs(lapse) > 0:  # need to check like that because otherwise 0.0 can trigger loop
    thlend = thl0 + lapse*zsize
    thl = np.arange(thl0, thlend, lapse*dz)
    print("Temperature profile with lapse rate.")
else:
    thl = [thl0] * kmax
    print("Constant temperature profile applied.")
    
# set inital linear wind profile
# is only correct for massflow forcing, not for uinf!

# uprof = np.linspace(0, 2*u, kmax)
# vprof = np.linspace(0, 2*v, kmax)
uinc = [(dzi/(newzsize - dzs[-1]))*u for dzi in dzs]
uprof = [round(np.sum(2*uinc[:it]), 8) for it in range(kmax)]
vinc = [(dzi/(newzsize - dzs[-1]))*v for dzi in dzs]
vprof = [np.sum(2*vinc[:it]) for it in range(kmax)]

# uprof = [(u*newzsize/kmax)/dzi for dzi in dzs]
# vprof = [(v*newzsize/kmax)/dzi for dzi in dzs]

# checks
uheights = [ui*dzi for ui, dzi in zip(uprof, dzs)]
ubulk = np.sum(uheights)/newzsize

vheights = [vi*dzi for vi, dzi in zip(vprof, dzs)]
vbulk = np.sum(vheights)/newzsize

print("Linear velocity profile applied.")
print("Average initial velocities u =", ubulk, "and v =", vbulk)
uflowrate = ubulk*ysize*newzsize
vflowrate = vbulk*ysize*newzsize
print("uflowrate: ", uflowrate, "and vflowrate: ", vflowrate)

profs = [[z, temp, qt0, u, v, tke] for z, temp, u, v in zip(zgrid, thl, uprof, vprof)]

fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(uprof, zgrid)
ax1.set_xlabel("Initial u velocity [m/s]")
ax1.set_ylabel("Height [m]")
ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(vprof, zgrid)
ax2.set_xlabel("Initial v velocity [m/s]")
ax2.set_ylabel("Height [m]")
plt.tight_layout()
plt.show()

#### Write new prof inp file

In [ ]:
da.write.profinp(profs, inpspath + 'prof.inp.' + expnr)

#### Update parameters in namoptions file

In [ ]:
profchanges = {'thl0': thl0, 'lapse': lapse, 
               'uflowrate': round(uflowrate), 'vflowrate': round(vflowrate)}
parameters = da.write.namoptions(parameters, profchanges, namoptions)

[Back to Contents](#Contents)

---
# Update scalars

In [ ]:
# set default values
sc1 = 0
sc2 = 0
sc3 = 0
sc4 = 0

scalars = [[z, sc1, sc2, sc3, sc4] for z in zgrid]

#### Write new scalar inp file

In [ ]:
da.write.scalarinp(scalars, inpspath + 'scalar.inp.' + expnr)

[Back to Contents](#Contents)

---
# Update blocks

#### Get blocks

In [ ]:
blocksfile = inpspath + 'blocks.inp.' + expnr
# blocksfile = inpspath + 'old-blocks.' + expnr
blocks = da.tools.getblocks(blocksfile)
print("blocks:\n", blocks)

## Change resolution/convert to indices

In [ ]:
# convert = np.array([imax/xsize, imax/xsize, jtot/ysize, jtot/ysize, kmax/zsize, kmax/zsize])
# block_indices = [block * convert for block in blocks]
# block_indices = [list([int(b) for b in block]) for block in block_indices]
# # block_indices2 = [[*block[0:4], block[4] - 1, block[5] - 1] for block in block_indices]
# print(block_indices)

## Show dimensionalised blocks

In [ ]:
dimblocks = da.tools.getdimblocks(blocks, xedge, yedge, zedge)
limits=[0, xsize, 0, ysize, 0, zsize/3]
# plot
fig = plt.figure(figsize=(3*2, 4))
# geometry plot
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
da.plot.blocks3d(dimblocks, ax=ax1, edgecolor='k', 
                   limits=limits)
ax1.xaxis.set_ticks(np.linspace(*limits[0:2], 5))
ax1.yaxis.set_ticks(np.linspace(*limits[2:4], 5))
ax1.zaxis.set_ticks(np.linspace(*limits[4:6], 4))
ax1.set_aspect('equal')
ax1.view_init(25, 235)
# layout plot
ax2 = fig.add_subplot(1, 2, 2)
da.plot.layout(dimblocks, ax=ax2, limits=limits[:4])
ax2.set_xticks(np.linspace(*limits[0:2], 5))
ax2.set_yticks(np.linspace(*limits[2:4], 5))
# plt.suptitle("Blocks")
plt.tight_layout()
plt.show()

## Block statistics

In [ ]:
blockstats = da.stats.blockstats(dimblocks, a0=xsize*ysize)
print("lamda_p = ", blockstats['lp'])
print("lambda_f = ", blockstats['lf'])
print("zmax = ", blockstats['zmax'])
print("zh = ", blockstats['zh'])
print("sigma_h = ", blockstats['sigma_h'])
print("no. of blocks: ", blockstats['nblocks'])
print("Blocks:\n", dimblocks)

### Update wall types

In [ ]:
walltypes_default = predir + 'default/walltypes.inp.xxx'
walltypes_new = inpspath + 'test-walltypes.inp.' + expnr
with open(walltypes_default, 'r') as infile, open(walltypes_new, 'a') as outfile:
    for line in infile:
            outfile.write(line)

## Make facets 

### NEED TO PUT IVO's SCRIPTS HERE

#### Write new blocks inp file

In [ ]:
da.write.blocksinp(blocks, inpspath + 'blocks.inp.' + expnr)  # NEEDS TO BE UPDATED

---
#### Update parameters in namoptions file after facets (MATLAB atm)

In [ ]:
facetchanges = {'nblocks': len(blocks), 'nfcts': 858}
parameters = da.write.namoptions(parameters, facetchanges, namoptions)

In [ ]:
pprint.pprint(parameters)

[Back to Contents](#Contents)